# Notebook 1

## Goals:
* Connect to the database
* query the tables in the database
* Exploration of data
* Activity: basic sql queries to explore the data


In [6]:
# import the libraries and packages

import os
import sys
import pandas as pd
import psycopg2

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.extract import connect_to_redshift

# using this library for reading password
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

## Connect to database

In [7]:
# import variables from .env file

dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [8]:
# Connecting to Redshift

connect = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


## Query the Tables

In [9]:
# Load and get an intial look at the cleaned online_transactions_cleaned data
query= """select * 
          from bootcamp.online_transactions_cleaned          
       """ 

online_trans_cleaned = pd.read_sql(query, connect)

In [10]:
online_trans_cleaned.head(5)

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536384,22189,CREAM HEART CARD HOLDER,3.95,4,15.8,2010-12-01 09:53:00,u18074,United Kingdom
1,536392,21891,TRADITIONAL WOODEN SKIPPING ROPE,1.25,12,15.0,2010-12-01 10:29:00,u13705,United Kingdom
2,536401,22068,BLACK PIRATE TREASURE CHEST,1.65,2,3.3,2010-12-01 11:21:00,u15862,United Kingdom
3,536412,22961,JAM MAKING SET PRINTED,1.45,24,34.8,2010-12-01 11:49:00,u1792,United Kingdom
4,536420,21889,WOODEN BOX OF DOMINOES,1.25,12,15.0,2010-12-01 12:03:00,u16583,United Kingdom


In [11]:
online_trans_cleaned.shape

(399841, 9)

In [20]:
online_trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            399841 non-null  object        
 1   stock_code         399841 non-null  object        
 2   description        399841 non-null  object        
 3   price              399841 non-null  float64       
 4   quantity           399841 non-null  int64         
 5   total_order_value  399841 non-null  float64       
 6   invoice_date       399841 non-null  datetime64[ns]
 7   customer_id        399841 non-null  object        
 8   country            399841 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.5+ MB


In [12]:
online_trans_cleaned.isnull()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
399836,False,False,False,False,False,False,False,False,False
399837,False,False,False,False,False,False,False,False,False
399838,False,False,False,False,False,False,False,False,False
399839,False,False,False,False,False,False,False,False,False


In [13]:
online_trans_cleaned.isnull().sum()

invoice              0
stock_code           0
description          0
price                0
quantity             0
total_order_value    0
invoice_date         0
customer_id          0
country              0
dtype: int64

## Explore the data by SQL Query

In [14]:
# 1.  How many invoices does the table contain?

query = """select count(invoice) as number_of_invoices
           from bootcamp.online_transactions_cleaned
        """
pd.read_sql(query, connect)

,number_of_invoices
0,399841


In [25]:
# Do invoices contain multiple records? 

query = """select count(Distinct invoice) as total_no_invoice
           from bootcamp.online_transactions_cleaned
           """
pd.read_sql(query, connect)

,total_no_invoice
0,21791


* It appears that the dataset comprises a total of 399,841 records, indicating individual transactions. However, when considering distinct invoices, there are 21,791 unique invoices. This suggests that some invoices have multiple associated records or lines within the dataset

In [21]:
# 2. When was the first and last purchase?

query = """select MIN(invoice_date) AS first_purchase,
           MAX(invoice_date) AS last_purchase,
           MAX(invoice_date) - MIN(invoice_date) AS total_days
           from bootcamp.online_transactions_cleaned 
        """
pd.read_sql(query, connect)

,first_purchase,last_purchase,total_days
0,2010-12-01 08:26:00,2011-12-09 12:50:00,373 days 04:24:00


In [22]:
# 3.How many customers does the table contain?

query = """select count(Distinct customer_id) As total_customers
           from bootcamp.online_transactions_cleaned
        """
pd.read_sql(query, connect)

,total_customers
0,4363


In [24]:
# 4. How many different types of stocks did the customer purchase?

query = """select count(Distinct stock_code) as num_of_stock_types
           from bootcamp.online_transactions_cleaned
        """
pd.read_sql(query, connect)

,num_of_stock_types
0,3679


In [26]:
# 5. What is the most popular stock? (You can look at top 10 sold items - does this differ across markets?)

query = """select stock_code, description, country, SUM(quantity) AS total_sold_quantity, 
           SUM(total_order_value) AS total_revenue 
           from bootcamp.online_transactions_cleaned
           group by stock_code, description, country
           order by total_sold_quantity desc
           limit 10
        """
pd.read_sql(query, connect)

,stock_code,description,country,total_sold_quantity,total_revenue
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,United Kingdom,47886,11856.04
1,22197,POPCORN HOLDER,United Kingdom,45194,34090.58
2,85099B,JUMBO BAG RED RETROSPOT,United Kingdom,40777,75236.43
3,84879,ASSORTED COLOUR BIRD ORNAMENT,United Kingdom,32580,52147.56
4,85123A,CREAM HANGING HEART T-LIGHT HOLDER,United Kingdom,32136,88416.20
5,22616,PACK OF 12 LONDON TISSUES,United Kingdom,24297,6920.49
6,17003,BROCADE RING PURSE,United Kingdom,22672,5718.69
7,21212,PACK OF 72 RETROSPOT CAKE CASES,United Kingdom,22182,10856.22
8,22178,VICTORIAN GLASS HANGING T-LIGHT,United Kingdom,21427,27350.31
9,21977,PACK OF 60 PINK PAISLEY CAKE CASES,United Kingdom,19882,9438.00


In [35]:
# What were the top performing stocks per country based on quantity sold?

query = """select country, stock_code, qty_sold
        From (
            Select 
               country, 
               stock_code, 
               SUM(quantity) as qty_sold,
               RANK() OVER(Partition by country ORDER BY SUM(quantity) DESC) as rank
        from bootcamp.online_transactions_cleaned
        group by country, stock_code) as sold_per_country
        where rank = 1
        Order by qty_sold DESC, country;
        """

pd.read_sql(query, connect)

,country,stock_code,qty_sold
0,United Kingdom,84077,47886
1,Netherlands,23084,4801
2,France,23084,3999
3,Japan,23084,3401
4,Australia,22492,2916
...,...,...,...
57,Saudi Arabia,22915,12
58,Saudi Arabia,22556,12
59,Saudi Arabia,22969,12
60,Saudi Arabia,22555,12


In [26]:
# Which are the top 10 customers based on their total revenue contribution?

query = """select customer_id,
           SUM(total_order_value) AS total_revenue 
           from bootcamp.online_transactions_cleaned
           group by customer_id
           order by total_revenue desc
           limit 10
        """
pd.read_sql(query, connect)

,customer_id,total_revenue
0,u14646,278778.02
1,u18102,259657.30
2,u1745,189575.53
3,u14911,132893.24
4,u12415,123638.18
5,u14156,114335.77
6,u17511,88138.20
7,u16684,65920.12
8,u14096,65164.79
9,u13694,62961.54


In [27]:
# 6. What is the average order value i.e. price * quantity?

query = """select avg(total_order_value) As avg_order_value
           from bootcamp.online_transactions_cleaned
        """
pd.read_sql(query, connect)

,avg_order_value
0,20.716904


In [30]:
# Which are the top 10 customers based on their average revenue?

query = """select customer_id,
           ROUND(Sum(total_order_value)/Count(Distinct invoice)) as avg_revenue
           from bootcamp.online_transactions_cleaned
           group by customer_id
           order by avg_revenue desc
           Limit 10
        """
pd.read_sql(query,connect)

,customer_id,avg_revenue
0,u15098,9905.0
1,u12357,6208.0
2,u15749,5384.0
3,u12415,5152.0
4,u12688,4874.0
5,u1259,4669.0
6,u12752,4367.0
7,u18102,4328.0
8,u18251,4315.0
9,u16,4131.0


In [37]:
# How many orders were placed per month?

query = """ Select
           CAST(DATE_PART(Year, invoice_date) as int) as year,
           CAST(DATE_PART(Month, invoice_date) as int) as month,
           Count(Distinct invoice) as no_invoices
           from bootcamp.online_transactions_cleaned
           group by year, month
           order by year, month
        """
pd.read_sql(query, connect)

,year,month,no_invoices
0,2010,12,1692
1,2011,1,1225
2,2011,2,1181
3,2011,3,1588
4,2011,4,1358
5,2011,5,1808
6,2011,6,1686
7,2011,7,1555
8,2011,8,1506
9,2011,9,2038


* Note:

- In January 2011, 1,225 invoices were issued, whereas in November 2011, there was a substantial increase in the number of invoices, reaching 3,034. This dataset can be valuable for analyzing invoice trends over time and understanding potential seasonal variations in business activities.

- December 2010 saw a significantly higher volume of invoices, totaling 1,692, compared to December 2011, which had 912 invoices. This indicates a noticeable decline in the number of invoices issued between December 2010 and December 2011.

In [38]:
# What was the total monthly revenue?

query = """ Select
            CAST(DATE_PART(YEAR, invoice_date) as int) as year,
            CAST(DATE_PART(MONTH, invoice_date) as int) as month,
            ROUND(SUM(total_order_value), 2) as total_revenue
            From bootcamp.online_transactions_cleaned
            Group by year, month
            Order by year, month
        """
pd.read_sql(query, connect)

,year,month,total_revenue
0,2010,12,548443.92
1,2011,1,471580.34
2,2011,2,434218.17
3,2011,3,573838.05
4,2011,4,421527.77
5,2011,5,650735.39
6,2011,6,641129.21
7,2011,7,580714.73
8,2011,8,612966.30
9,2011,9,924390.57


In [45]:
# How does the monthly revenue in 2011 compare to the average revenue?

query = """ Select year, month,total_revenue,
            ROUND(Avg(total_revenue) OVER ()) as avg_revenue,
            ROUND(total_revenue - AVG(total_revenue) OVER ()) as diff
            From (
                 Select
                 CAST(DATE_PART(YEAR, invoice_date) as int) as year,
                 CAST(DATE_PART(MONTH, invoice_date) as int) as month,
                 ROUND(SUM(total_order_value), 2) as total_revenue
                 From bootcamp.online_transactions_cleaned
                 Group by year, month) rev_table
                 Where year = 2011
                 Order by month;
         """
pd.read_sql(query, connect)

,year,month,total_revenue,avg_revenue,diff
0,2011,1,471580.34,644585.0,-173005.0
1,2011,2,434218.17,644585.0,-210367.0
2,2011,3,573838.05,644585.0,-70747.0
3,2011,4,421527.77,644585.0,-223058.0
4,2011,5,650735.39,644585.0,6150.0
5,2011,6,641129.21,644585.0,-3456.0
6,2011,7,580714.73,644585.0,-63871.0
7,2011,8,612966.30,644585.0,-31619.0
8,2011,9,924390.57,644585.0,279805.0
9,2011,10,965151.25,644585.0,320566.0


In [53]:
# How many orders were made for each country?

query = """ select country, count(Distinct invoice) as total_no_invoice
            from bootcamp.online_transactions_cleaned
            group by country
            order by total_no_invoice desc
        """
            
pd.read_sql(query,connect)

,country,total_no_invoice
0,United Kingdom,19583
1,Germany,578
2,France,439
3,EIRE,313
4,Belgium,117
5,Spain,100
6,Netherlands,97
7,Australia,67
8,Switzerland,65
9,Portugal,55


In [63]:
# From which country did we generate the most and least revenue?

query = """ Select country,total_revenue,
                   CASE WHEN rank_asc = 1 THEN 'least_revenue' ELSE 'top_revenue' END AS performance
            from (
            Select 
              country,
              ROUND(SUM(total_order_value), 2) as total_revenue,
              DENSE_RANK() OVER (ORDER BY SUM(total_order_value)) as rank_asc,
              DENSE_RANK() OVER (ORDER BY SUM(total_order_value) DESC) as rank_desc
              from bootcamp.online_transactions_cleaned
              group by country) as ranking
              where 1 in (rank_asc, rank_desc)
        """

pd.read_sql(query, connect)

,country,total_revenue,performance
0,United Kingdom,6815375.09,top_revenue
1,Saudi Arabia,131.17,least_revenue


In [65]:
# What is the most popular and least popular product?

query = """ Select stock_code,description,total_sold_qty,
                   CASE WHEN rank_asc = 1 THEN 'least_popular' ELSE 'most_popular' END AS popularity
            from (
            Select 
               stock_code,
               description,
               SUM(quantity) as total_sold_qty,
               DENSE_RANK() OVER (ORDER BY SUM(quantity)) as rank_asc,
               DENSE_RANK() OVER (ORDER BY SUM(quantity) DESC) as rank_desc
            from bootcamp.online_transactions_cleaned
            group by stock_code, description) as ranking
            Where 1 in (rank_asc, rank_desc)
            """
pd.read_sql(query, connect)

,stock_code,description,total_sold_qty,popularity
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53119,most_popular
1,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-1475,least_popular


* The product with stock_code 84077 ("WORLD WAR 2 GLIDERS ASSTD DESIGNS") has sold a significant number of units, with a total sold quantity of 53,119. This indicates that it's a popular item.
* The product with stock_code 84347 ("ROTATING SILVER ANGELS T-LIGHT HLDR") has a negative value for total sold quantity (-1,475). This suggests that there might be a data issue or that more of these items were returned or canceled than were sold, making it the least popular item in terms of sales.

In [66]:
# Display the data for records where the 'quantiy' column indicates a quantity less than 0?

query = """ Select *
            from bootcamp.online_transactions_cleaned
            where quantity < 0
            Order by quantity
            Limit 10
        """
pd.read_sql(query,connect)

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",2.08,-80995,-168469.60,2011-12-09 09:27:00,u16446,United Kingdom
1,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.60,2011-01-18 10:17:00,u12346,United Kingdom
2,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,0.03,-9360,-280.80,2010-12-02 14:23:00,u15838,United Kingdom
3,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,2.10,-3114,-6539.40,2011-04-18 13:08:00,u15749,United Kingdom
4,C550456,21175,GIN AND TONIC DIET METAL SIGN,1.85,-2000,-3700.00,2011-04-18 13:08:00,u15749,United Kingdom
5,C550456,85123A,CREAM HANGING HEART T-LIGHT HOLDER,2.55,-1930,-4921.50,2011-04-18 13:08:00,u15749,United Kingdom
6,C556522,22920,HERB MARKER BASIL,0.55,-1515,-833.25,2011-06-13 11:21:00,u16938,United Kingdom
7,C550456,47566B,TEA TIME PARTY BUNTING,2.55,-1300,-3315.00,2011-04-18 13:08:00,u15749,United Kingdom
8,C570556,20971,PINK BLUE FELT CRAFT TRINKET BOX,1.06,-1296,-1373.76,2011-10-11 11:10:00,u16029,United Kingdom
9,C569552,15034,PAPER POCKET TRAVELING FAN,0.07,-1200,-84.00,2011-10-04 17:44:00,u14533,United Kingdom


In [68]:
# How many records in dataset have a negative quantity ?

query = """ Select Count(Distinct invoice) As total_neg_qty
            from bootcamp.online_transactions_cleaned
            where quantity < 0 
        """
pd.read_sql(query,connect)

,total_neg_qty
0,3383


*  The presence of 3,383 invoices with negative quantities out of 21,791 invoices indicates a substantial issue that requires investigation and action. It's essential to understand the root causes and take steps to improve data accuracy, inventory management, customer satisfaction, and overall operational efficiency.

In [70]:
# 7. How many Stocks have the Description “Unknown”? How will you handle this when building customer segments?¶

query = """ select count(Distinct stock_code) As total_unknown_stocks
             from bootcamp.online_transactions_cleaned 
             where description = 'Unknown'
        """
pd.read_sql(query,connect)

,total_unknown_stocks
0,18


In [71]:
# Display the stock_codes that have Discription as 'Unknown'

query = """ select Distinct stock_code As total_unknown_stocks, description
            from bootcamp.online_transactions_cleaned 
            where description = 'Unknown'
            Order by stock_code
        """
pd.read_sql(query,connect)

,total_unknown_stocks,description
0,16151A,Unknown
1,16162M,Unknown
2,18007,Unknown
3,20964,Unknown
4,21703,Unknown
5,21704,Unknown
6,21705,Unknown
7,22686,Unknown
8,22878,Unknown
9,22889,Unknown
